https://www.kaggle.com/code/jieyima/netflix-recommendation-collaborative-filtering/notebook

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# %matplotlib inline
plt.style.use("ggplot")

import sklearn
from sklearn.decomposition import TruncatedSVD

In [ ]:
amazon_ratings = pd.read_csv('/content/drive/MyDrive/iisc/ratings_Beauty.csv')
amazon_ratings = amazon_ratings.dropna()
amazon_ratings.head()

,UserId,ProductId,Rating,Timestamp
0,A39HTATAQ9V7YF,0205616461,5.0,1369699200
1,A3JM6GV9MNOF9X,0558925278,3.0,1355443200
2,A1Z513UWSAAO0F,0558925278,5.0,1404691200
3,A1WMRR494NWEWV,0733001998,4.0,1382572800
4,A3IAAVS479H7M7,0737104473,1.0,1274227200


In [ ]:
amazon_ratings.shape

(2023070, 4)

In [ ]:
popular_products = pd.DataFrame(amazon_ratings.groupby('ProductId')['Rating'].count())
most_popular = popular_products.sort_values('Rating', ascending=False)
most_popular.head(10)

,Rating
ProductId,
B001MA0QY2,7533
B0009V1YR8,2869
B0043OYFKU,2477
B0000YUXI0,2143
B003V265QW,2088
B000ZMBSPE,2041
B003BQ6QXK,1918
B004OHQR1Q,1885
B00121UVU0,1838


In [ ]:
amazon_ratings1 = amazon_ratings.head(10000)

In [ ]:
ratings_utility_matrix = amazon_ratings1.pivot_table(values='Rating', index='UserId', columns='ProductId', fill_value=0)
ratings_utility_matrix.head()

ProductId,0205616461,0558925278,0733001998,0737104473,0762451459,1304139212,1304139220,130414089X,130414643X,1304146537,...,B000052YPE,B000052YPF,B000052YPG,B000052YPH,B000052YPM,B000052YPU,B000052YPV,B000052YPY,B000052YQ0,B000052YQ2
UserId,,,,,,,,,,,,,,,,,,,,,
A00205921JHJK5X9LNP42,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
A024581134CV80ZBLIZTZ,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
A03056581JJIOL5FSKJY7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
A03099101ZRK4K607JVHH,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
A0505229A7NSH3FRXRR4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
ratings_utility_matrix.shape

(9697, 886)

In [ ]:
X = ratings_utility_matrix.T

print(X.shape)
X.head()

(886, 9697)


UserId,A00205921JHJK5X9LNP42,A024581134CV80ZBLIZTZ,A03056581JJIOL5FSKJY7,A03099101ZRK4K607JVHH,A0505229A7NSH3FRXRR4,A05492663T95KW63BR75K,A059547920Q3LZVFHLPI3,A07410232KYRFR25CIUGJ,A082796624UNM47DSAI6K,A0864963DOAY7LXGS5I6,...,AZW1HXXYAC15B,AZWRTJPN7NXT,AZWTXHXZXFAYP,AZYQEFB9Y5N22,AZZHB6U54UDYW,AZZHJZP4GQPPZ,AZZNK89PXD006,AZZOFVMQC0BJG,AZZQXL8VDCFTV,AZZTJQ7CQZUD8
ProductId,,,,,,,,,,,,,,,,,,,,,
0205616461,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0558925278,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0733001998,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0737104473,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0762451459,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
Svd = TruncatedSVD(n_components=10)
decomposed_matrix = Svd.fit_transform(X)
decomposed_matrix.shape

(886, 10)

In [ ]:
correlation_matrix = np.corrcoef(decomposed_matrix)
correlation_matrix.shape

(886, 886)

recommend

if user brought product 6117036094 recommend him more related products

Content based

In [ ]:
product=X.index[99]
print(product)

6117036094


In [ ]:
i =product

product_names = list(X.index)
product_ID = product_names.index(i)
product_ID

99

In [ ]:
correlation_product_ID = correlation_matrix[product_ID]
correlation_product_ID.shape

(886,)

highly recommended items

In [ ]:
Recommend = list(X.index[correlation_product_ID > 0.90])

# Removes the item already bought by the customer
Recommend.remove(i)
Recommend = sorted(Recommend, reverse=True)
Recommend[0:9]

['B000052YPH',
 'B000052YOC',
 'B000052YJS',
 'B000052Y6L',
 'B000052Y3V',
 'B000052Y31',
 'B000052Y23',
 'B000052Y21',
 'B000052XYE']

In [ ]:
!pip install scikit-surprise


In [ ]:
import pandas as pd
import numpy as np
import math
import re
from scipy.sparse import csr_matrix
import matplotlib.pyplot as plt
import seaborn as sns
from surprise import Reader, Dataset, SVD
sns.set_style("ticks")

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/iisc/ratings_Beauty.csv')
df = df.dropna()
df.head()

,UserId,ProductId,Rating,Timestamp
0,A39HTATAQ9V7YF,0205616461,5.0,1369699200
1,A3JM6GV9MNOF9X,0558925278,3.0,1355443200
2,A1Z513UWSAAO0F,0558925278,5.0,1404691200
3,A1WMRR494NWEWV,0733001998,4.0,1382572800
4,A3IAAVS479H7M7,0737104473,1.0,1274227200


In [ ]:
reader = Reader()

# get just top 100K rows for faster run time
data = Dataset.load_from_df(df[['UserId', 'ProductId', 'Rating']], reader)


In [ ]:
print(data)

In [ ]:
df_user = df[(df['UserId'] == 'A1WMRR494NWEWV') & (df['Rating'] == 5)]
df_user = df_user.set_index('ProductId')

print(df_user)

Empty DataFrame
Columns: [UserId, Rating, Timestamp]
Index: []


In [ ]:
svd = SVD()
trainset = data.build_full_trainset()
svd.fit(trainset)

In [ ]:
from surprise import Reader, Dataset, SVD
from surprise.model_selection import train_test_split

# Load your dataset (replace 'df' with your actual DataFrame)
reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(df[['UserId', 'ProductId', 'Rating']], reader)

# Split the data into a training set and a test set
trainset, testset = train_test_split(data, test_size=0.2)

# Initialize the SVD algorithm and train it on the training set
svd = SVD()
svd.fit(trainset)

# Make predictions on the test set
predictions = svd.test(testset)



In [ ]:
import pandas as pd
from surprise import Dataset, Reader
from surprise.model_selection import train_test_split
from surprise import accuracy

# Assuming you have a DataFrame 'df' with 'user_id', 'item_id', and 'rating' columns

# Create a Surprise Dataset from the DataFrame

# Split the data into a training set and a test set
trainset, testset = train_test_split(data, test_size=0.2, random_state=42)

# Calculate the average rating in the training set
def calculate_average_rating(trainset):
    total_rating = 0
    total_users = 0
    for uid in trainset.all_users():
        user_ratings = trainset.ur[uid]
        total_rating += sum([r for (_, r) in user_ratings])
        total_users += len(user_ratings)
    return total_rating / total_users

average_rating = calculate_average_rating(trainset)

# Create a list of baseline predictions (average rating) for all test set items
baseline_predictions = [average_rating] * len(testset)

# Extract actual ratings from the test set
actual_ratings = [rating for (_, _, rating) in testset]

# Calculate MAE for the baseline predictions
baseline_mae = accuracy.mae(baseline_predictions, actual_ratings)

print("Baseline MAE:", baseline_mae)


In [ ]:
mae_sum = sum([abs(actual_rating - baseline_rating) for actual_rating, baseline_rating in zip(actual_ratings, baseline_predictions)])
baseline_mae = mae_sum / len(actual_ratings)
mse_sum = sum([(actual_rating - baseline_rating) ** 2 for actual_rating, baseline_rating in zip(actual_ratings, baseline_predictions)])
baseline_mse = mse_sum / len(actual_ratings)

print("Baseline MSE:", baseline_mse)
print("Baseline MAE:", baseline_mae)

Baseline MSE: 1.7171722008410564
Baseline MAE: 1.0499626964650945


In [ ]:
from surprise import accuracy

# Calculate RMSE (Root Mean Squared Error)
rmse = accuracy.rmse(predictions)

# Calculate MAE (Mean Absolute Error)
mae = accuracy.mae(predictions)

print("RMSE:", rmse)
print("MAE:", mae)


RMSE: 1.2477
MAE:  0.9758
RMSE: 1.2477423266869503
MAE: 0.9758243616187823


prediction of user A1Z513UWSAAO0F new prodcts based on ratings prediction

In [ ]:
uid='A1Z513UWSAAO0F'

In [ ]:
user= pd.DataFrame()
user['ProductId']=df['ProductId']

In [ ]:
user['Estimate_Score']  = user['ProductId'].apply(lambda x: svd.predict(uid, x).est)

In [ ]:
user = user.sort_values('Estimate_Score', ascending=False)

In [ ]:
user = user.drop_duplicates()

In [ ]:
user.head(10)

,ProductId,Estimate_Score
1206978,B004CNRDBU,4.948551
1910148,B00DIA8EPC,4.930674
204309,B000C1ZA0C,4.922456
349735,B000NZAIC2,4.904121
1547861,B00716H7U2,4.894233
1327162,B004Y9GL10,4.892648
41701,B0000Y3LKM,4.891085
955193,B002YFN49I,4.888070
1551923,B0073B3ESY,4.884755
1075472,B003NSABXW,4.877442
